In [1]:
from powersimdata.input.grid import Grid
from prereise.gather.winddata.rap import rap, helpers # impute

import pandas as pd
from matplotlib import pyplot
import numpy as np
import utm
import math
import pickle

from scipy.spatial.distance import cdist

Wind Farms in Network + Branches/lines & buses

In [2]:
grid = Grid(["Texas"])

Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv


In [3]:
buses = grid.bus

In [4]:
#  X and Y coordinates of buses 
result = utm.from_latlon(buses.lat.to_numpy(),buses.lon.to_numpy())
buses['X'], buses['Y'] = result[0], result[1]

In [5]:
buses.head()

,type,Pd,Qd,Gs,Bs,zone_id,Vm,Va,baseKV,loss_zone,...,Vmin,lam_P,lam_Q,mu_Vmax,mu_Vmin,interconnect,lat,lon,X,Y
bus_id,,,,,,,,,,,,,,,,,,,,,
3001001,1,20.78,5.89,0,0.0,301,0.998372,-22.3462,115.0,9,...,0.9,0.0,0,0,0,Texas,31.9067,-102.262,758924.088679,3.533366e+06
3001002,1,15.41,4.37,0,0.0,301,1.022910,-17.8585,115.0,9,...,0.9,0.0,0,0,0,Texas,29.8880,-104.519,546443.618177,3.306472e+06
3001003,1,0.00,0.00,0,0.0,301,1.014100,-17.5032,115.0,9,...,0.9,0.0,0,0,0,Texas,32.9264,-101.648,813463.290141,3.648115e+06
3001004,2,0.00,0.00,0,0.0,301,1.020050,-19.3247,230.0,9,...,0.9,0.0,0,0,0,Texas,32.9264,-101.648,813463.290141,3.648115e+06
3001005,1,0.00,0.00,0,0.0,301,1.012380,-15.9276,115.0,9,...,0.9,0.0,0,0,0,Texas,32.2075,-101.388,840500.219088,3.569161e+06


In [6]:
branches = grid.branch

In [7]:
branches.columns

Index(['from_bus_id', 'to_bus_id', 'r', 'x', 'b', 'rateA', 'rateB', 'rateC',
       'ratio', 'angle', 'status', 'angmin', 'angmax', 'Pf', 'Qf', 'Pt', 'Qt',
       'mu_Sf', 'mu_St', 'mu_angmin', 'mu_angmax', 'branch_device_type',
       'interconnect', 'from_zone_id', 'to_zone_id', 'from_zone_name',
       'to_zone_name', 'from_lat', 'from_lon', 'to_lat', 'to_lon'],
      dtype='object')

In [8]:
branches.shape

(3206, 31)

In [9]:
# Assign start and end X and Y coordinates
from_result = utm.from_latlon(branches.from_lat.to_numpy(),branches.from_lon.to_numpy())
branches['fromX'], branches['fromY'] = from_result[0], from_result[1]

to_result = utm.from_latlon(branches.to_lat.to_numpy(),branches.to_lon.to_numpy())
branches['toX'], branches['toY'] = to_result[0], to_result[1]


In [10]:
branches.head()

,from_bus_id,to_bus_id,r,x,b,rateA,rateB,rateC,ratio,angle,...,from_zone_name,to_zone_name,from_lat,from_lon,to_lat,to_lon,fromX,fromY,toX,toY
branch_id,,,,,,,,,,,,,,,,,,,,,
100915,3001001,3001064,0.00524,0.03580,0.00609,221.0,0.0,0.0,0.0,0.0,...,Far West,Far West,31.9067,-102.262,32.0011,-102.312,758924.088679,3.533366e+06,753934.073875,3.543716e+06
100916,3001001,3001064,0.00524,0.03580,0.00609,221.0,0.0,0.0,0.0,0.0,...,Far West,Far West,31.9067,-102.262,32.0011,-102.312,758924.088679,3.533366e+06,753934.073875,3.543716e+06
100917,3001001,3001071,0.00435,0.02808,0.00536,221.0,0.0,0.0,0.0,0.0,...,Far West,Far West,31.9067,-102.262,31.8403,-102.326,758924.088679,3.533366e+06,753051.428019,3.525851e+06
100918,3001001,3001071,0.00435,0.02808,0.00536,221.0,0.0,0.0,0.0,0.0,...,Far West,Far West,31.9067,-102.262,31.8403,-102.326,758924.088679,3.533366e+06,753051.428019,3.525851e+06
100919,3001002,3001007,0.07776,0.46693,0.06336,98.0,0.0,0.0,0.0,0.0,...,Far West,Far West,29.8880,-104.519,31.0935,-104.625,546443.618177,3.306472e+06,535764.915208,3.440025e+06


In [11]:
# Merge lists of x and y coordinates into list of coordinate pair tuples
def merge(list1, list2):
    return list(map(lambda x, y:(x,y), list1, list2))

def sample_points(x1,y1,x2,y2,d):
    # Start: (x1,y1)
    # End: (x2,y2)
    # d: step size

    total_dist = math.sqrt((y2-y1)**2 + (x2-x1)**2)
    num_points = int(total_dist//d)
   
    # Handle cases with vertical (slope = inf) or horizontal (slope = 0)
    eps = 0.1
    if abs(y1 - y2) < eps: # horizontal
        x_vals = np.linspace(start=x1,stop=x2,num=num_points)
        y_vals = np.ones(num_points)*y1
    elif abs(x1 - x2) < eps: # vertical
        x_vals = np.ones(num_points)*x1
        y_vals = np.linspace(start=y1,stop=y2,num=num_points)
    else:
        slope = (y2 - y1)/(x2 - x1)
        dx = d/math.sqrt(1+slope**2)
        dy = slope*dx

        x_vals = np.zeros(num_points)
        y_vals = np.zeros(num_points)

        # Start point
        x_vals[0] = x1
        y_vals[0] = y1 

        # End point
        x_vals[num_points-1] = x2
        y_vals[num_points-1] = y2

        for i in range(1,num_points-1): 
            x_vals[i] = x_vals[i-1] + dx
            y_vals[i] = y_vals[i-1] + dy

    return merge(x_vals,y_vals)

In [12]:
# Matching sampled (x,y) point along branch to nearest available weather datapoint
def closest_point(point,weather_points):
    min_index = cdist([point],weather_points).argmin()
    return min_index, weather_points[min_index]

In [13]:
# Loop through all branches and store the closest weather points - Only need to do this ONCE
# Same fixed points sampled throughout
num_branches = branches.shape[0]
branch_weather_points = {}
branch_weather_indices = {}

mac = False

if mac:
    file_path = '/Users/vinee/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/MIT/Semesters/Spring 2022/15.S08/Project/weather_data_Jan2016.pkl'
else:
    file_path = "C:\\Users\\vinee\\OneDrive - Massachusetts Institute of Technology\\MIT\\Semesters\\Spring 2022\\15.S08\\Project\\weather_data_Jan2016.pkl"

with open(file_path, 'rb') as file:
        x_orig_all = pickle.load(file)
        y_orig_all = pickle.load(file)

# Need to convert weather (x,y) coordinates from Lambert to utm projection
weather_points = merge(x_orig_all[1],y_orig_all[1])

In [14]:
d = 1000 # units in km?

for i in range(num_branches):
    
    branch_sample_points = sample_points(branches.iloc[i].fromX,branches.iloc[i].fromY,branches.iloc[i].toX,branches.iloc[i].toY,d)
    branch_weather_indices[i] = [closest_point(pt,weather_points)[0] for pt in branch_sample_points] 
    branch_weather_points[i] = [closest_point(pt,weather_points)[1] for pt in branch_sample_points]

In [15]:
len(branch_weather_points.keys())

3206

In [ ]:
# Loop through all hours
# Load weather data
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
days_per_month = [31,29,31,30,31,30,31,31,30,31,30,31]
hours_per_month = days_per_month * 60 # h indexing
total_hours = sum(hours_per_month)

str1 = "C:\\Users\\vinee\\OneDrive - Massachusetts Institute of Technology\\MIT\\Semesters\\Spring 2022\\15.S08\\Project\\weather_data_"
str2 = "2016.pkl"

dlr_values = np.zeros(num_branches,total_hours)

for i in range(len(months)):
    file_name = str1 + months[i] + str2

    with open(file_name, 'wb') as file:
            x_orig_all = pickle.load(file)
            y_orig_all = pickle.load(file)
            temp_all = pickle.load(file)
            u_all = pickle.load(file)
            v_all = pickle.load(file)
            times = pickle.load(file)

    weather_points = merge(x_orig_all,y_orig_all)

    d = 100 # units in km?

    for i in range(num_branches):
        branch_sample_points = sample_points(branches.fromX,branches.fromY,branches.toX,branches.toY,d)
        weather_indices = [closest_point(pt,weather_points)[0] for pt in branch_sample_points] 
        weather_points = [closest_point(pt,weather_points)[1] for pt in branch_sample_points] 
        u = u_all